In [1]:
from solid.utils import *  # Not required, but the utils module is useful
from solid import *
import subprocess
import platform
import pathlib
    
# move two directories up and import mycad utilities
%cd ../..
from mycad.mycad import *
%cd -0
    
# Fn is for number of segments in a circle to render to. Check openscad for other options
FN = 4*32

FUDGE = 1e-4

u:\Syncthing\Work\CAD\git-labtools


u:\Syncthing\Work\CAD\git-labtools\mycad\mycad.py:188: UserWarning: Could not find OpenSCAD executable. Preview and STL renderings will fail.
  warnings.warn("Could not find OpenSCAD executable. Preview and STL renderings will fail.")


u:\Syncthing\Work\CAD\git-labtools\designs\circular_clamp


In [16]:
#======================================
# # Flashlight
# INNER_DIAM = 33
# RING_THICKNESS = 3
# RING_HEIGHT = 13
# SEPARATION = 1

# THROUGHHOLE_DIAM = 2.7+0.7
# WRENCH_SIZE = 5.3+0.5
# NUT_SEP = 1
# COUNTER_THICKNESS = 1
# HEAD_DIAM = 5.2+0.5

# CYLATTACH_LENGTH = 7
#======================================
# Flashlight 18650
INNER_DIAM = 28.2
RING_THICKNESS = 3
RING_HEIGHT = 13
SEPARATION = 1

THROUGHHOLE_DIAM = 2.7+0.7
WRENCH_SIZE = 5.3+0.5
NUT_SEP = 1
COUNTER_THICKNESS = 1
HEAD_DIAM = 5.2+0.5

CYLATTACH_LENGTH = 7

#======================================
# # # telephoto
# INNER_DIAM = 77.1 - 1 # telephoto
# RING_THICKNESS = 3
# RING_HEIGHT = 9
# SEPARATION = 1

# THROUGHHOLE_DIAM = 2.7+0.7
# WRENCH_SIZE = 5.3+0.5
# NUT_SEP = 1
# COUNTER_THICKNESS = 1
# HEAD_DIAM = 5.2+0.5

# CYLATTACH_LENGTH = 7

#======================================
# # RPI CAM
# INNER_DIAM = 34.9 - 1
# RING_THICKNESS = 3
# RING_HEIGHT = 3.4
# SEPARATION = 1

# THROUGHHOLE_DIAM = 2.7+0.7
# WRENCH_SIZE = 5.3+0.5
# NUT_SEP = 3
# COUNTER_THICKNESS = 1
# HEAD_DIAM = 5.2+0.5

# CYLATTACH_LENGTH = 7

In [17]:

psphere = pointed_sphere()
sketch = circle(d=INNER_DIAM+2*RING_THICKNESS) - circle(d=INNER_DIAM)
ring = linear_extrude(RING_HEIGHT)(sketch)


ndiam = nut_diam(WRENCH_SIZE)
hex_hole = cylinder(d=ndiam, h=INNER_DIAM, segments=6)
hex_hole = up(SEPARATION/2+COUNTER_THICKNESS)(hex_hole)

head_hole = mirror([0,0,1])(cylinder(d=HEAD_DIAM, h=INNER_DIAM))
head_hole = down(SEPARATION/2+COUNTER_THICKNESS)(head_hole)

throughhole = cylinder(d=THROUGHHOLE_DIAM, h=INNER_DIAM, center=True)

holes = hex_hole + head_hole + throughhole

ca_diam = ndiam*1.2
cyl_attach = cylinder(d=ca_diam, h=CYLATTACH_LENGTH, center=True)
tmpcube = my_cube([ca_diam/2, ca_diam, CYLATTACH_LENGTH], center=[0,1,1])
cyl_attach = hull()(cyl_attach + tmpcube)

cyl_capture = cube([20,10,100], center=True)


def to_edge(piece):
    piece = rotate([0,90,0])(piece)
    piece = forward(INNER_DIAM/2+ndiam/2+NUT_SEP)(piece)
    piece = up(ca_diam/2)(piece)
    return piece

holes = to_edge(holes)
cyl_attach = to_edge(cyl_attach)
cyl_capture = to_edge(cyl_capture)

cyl_attach = hull()(cyl_attach + cyl_capture*ring)

ring += cyl_attach
ring -= holes
ring -= cylinder(d=INNER_DIAM, h=INNER_DIAM)#extra punch

# TODO: rounding correction, psphere
ring -= my_cube([SEPARATION, 5*INNER_DIAM, 5*INNER_DIAM], center=[1,0,1])

my_render(ring)

In [18]:
POST_DIAM = 25.4/2-0.7
POST_HEIGHT = 60
# POST_HEIGHT = 40
INSET = 1
HULLBIT_FACTOR = 0.5+0.5/np.sqrt(2) # how high the hull is compared to post

post = rotate([90,0,0])(cylinder(d=POST_DIAM, h=POST_HEIGHT))
post = up(POST_DIAM/2-INSET)(post) - mirror([0,0,1])(my_cube(1000, center=[1,1,0]))

INNER_RING_THICKNESS = 2 # how much to leave to the oiter ring
outer_ring = cylinder(d=INNER_DIAM+2*RING_THICKNESS, h=RING_HEIGHT) - cylinder(d=INNER_DIAM+2*INNER_RING_THICKNESS, h=RING_HEIGHT)

hullbit = mirror([0,1,0])(my_cube([5,100,HULLBIT_FACTOR*POST_DIAM-INSET], center=[1,0,0]))*outer_ring

POST_DIST = 10
post = back(INNER_DIAM/2+POST_DIST)(post)

post = hull()(post+hullbit)

comp_clamp = ring+post
my_render(comp_clamp)

In [19]:
# distance_plate = part
parts = [
         ("cric_clamp", comp_clamp),
        ]

for partname, part in parts:
    scadname = partname+".scad"
    stlname = partname+".stl"

    #make scad
    scad_render_to_file(part, scadname, file_header=f"$fn = {FN};")

    #render scad to stl
    comm = [OPENSCAD_EXEC, "-o", stlname, scadname]
    subprocess.run(comm)